## Remote Procedure Calls - Server and Client Code Only
### [Emil Sekerinski](http://www.cas.mcmaster.ca/~emil/), McMaster University, February 2022

It is recommended that you run this notebook on a server, which can be a remote server or your own computer; you have to comment out and adapt the server code below correspondingly. Note that under Windows, the Windows Defender may aks your for permission.

### Python GCD Server

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer

def gcd(x,y):
    while x != y:
        if x > y: x -= y
        else: y -= x
    return x

#server = SimpleXMLRPCServer(("your.server", 8020)) # use this if running remotely
server = SimpleXMLRPCServer(("localhost", 8020)) # use this if running locally
server.register_function(gcd, "gcd")
server.serve_forever()

### Python GCD Client

In [ ]:
import xmlrpc.client

#server = xmlrpc.client.ServerProxy("http://your.server:8020") # use this if running remotely
server = xmlrpc.client.ServerProxy("http://localhost:8020") # use this if running locally
server.gcd(81,36)

### Python Counter Server

In [ ]:
from xmlrpc.server import SimpleXMLRPCServer

class Counter:
    def __init__(self):
        self.a, self.e = 0, True
        # e == even(a)
    def inc(self):
        self.a, self.e = self.a + 1, not self.e
    def even(self):
        return self.e

# server = SimpleXMLRPCServer(("your.server", 8020), allow_none=True) # use this if running remotely
server = SimpleXMLRPCServer(("localhost", 8020), allow_none=True) # use this if running locally
server.register_instance(Counter()) # create Counter object, then register
server.serve_forever()

### Python Counter Client

In [ ]:
import xmlrpc.client

# c = xmlrpc.client.ServerProxy("http://your.server:8020") # use this if running remotely
c = xmlrpc.client.ServerProxy("http://localhost:8020") # use this if running locally
c.inc(); c.even()

In [ ]:
c.inc(); c.even()

### Go GCD Server

In [ ]:
%%writefile arithserver.go
package main
import ("net"; "net/rpc"; "net/http")

type GcdArgs struct{X, Y int}
type Arith int

func (a *Arith) Gcd(args *GcdArgs, result *int) error {
    // println(&t)
    x, y := args.X, args.Y
    for x != y {
        if x > y {x -= y} else {y -= x}
    }
    *result = x
    return nil
}
func main(){
    rpc.Register(new(Arith))
    rpc.HandleHTTP()
    l, err := net.Listen("tcp", ":8020")
    if err != nil {panic(err)}
    http.Serve(l, nil)
}

In [ ]:
!go run arithserver.go

In [ ]:
!netstat -np TCP | findstr "8020"

### Go GCD Client

In [ ]:
%%writefile arithclient.go
package main
import ("net/rpc")

type GcdArgs struct{X, Y int}

func main() {
    // client, err := rpc.DialHTTP("tcp", "your.server:8020")
    client, err := rpc.DialHTTP("tcp", "localhost:8020")
    if err != nil {panic(err)}
    
    // synchronous call
    var result int
    err = client.Call("Arith.Gcd", &GcdArgs{81, 36}, &result)
    if err != nil {panic(err)}
    println(result)
    
    // asynchronous call
    gcdCall := client.Go("Arith.Gcd", &GcdArgs{10, 4}, &result, nil)
    if gcdCall.Error != nil {panic(err)}
    <-gcdCall.Done //
    if gcdCall.Error != nil {panic(err)}
    println(*gcdCall.Reply.(*int)) // type assertion
}

In [ ]:
!go run arithclient.go

### Go Counter Server

In [3]:
%%writefile counterserver.go
package main
import ("net"; "net/rpc"; "net/http")

type Counter struct{A int32; E bool} // E == even(A)

type Void struct{}

func (self *Counter) Inc(args *Void, result *Void) error {
    self.A += 1; self.E = !self.E; return nil
}
func (self *Counter) Even(args *Void, result *bool) error {
    *result = self.E; return nil
}
func main(){
    c := new(Counter); c.A, c.E = 0, true
    rpc.Register(c); rpc.HandleHTTP()
    l, err := net.Listen("tcp", ":8021")
    if err != nil {panic(err)}
    http.Serve(l, nil)
}

Overwriting counterserver.go


In [ ]:
!go run counterserver.go

### Go Counter Client

In [ ]:
%%writefile counterclient.go
package main
import ("net/rpc")

type Void struct{}

func init_counter() *rpc.Client {
    // client, err := rpc.DialHTTP("tcp", "your.server:8020")
    client, err := rpc.DialHTTP("tcp", "localhost:8020")
    if err != nil {panic(err)}
    return client
}
func inc_counter(client *rpc.Client) {
    var result Void
    err := client.Call("Counter.Inc", &Void{}, &result)
    if err != nil {panic(err)}
}
func even_counter(client *rpc.Client) bool {
    var even bool
    err := client.Call("Counter.Even", &Void{}, &even)
    if err != nil {panic(err)}
    return even
}
func main(){
    c := init_counter()
    inc_counter(c); println(even_counter(c))
    inc_counter(c); println(even_counter(c))
}

In [ ]:
!go run counterclient.go